# 02-EDA

Neste notebook, é realizada a leitura dos dados originais, tratamento e exportação para a base que será submetida a análises e ao treinamento dos modelos.

# Importações

In [1]:
# Bibliotecas padrão
import pickle

# Bibliotecas utilitárias de terceiros

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import plotly.io as pio
import plotly.graph_objects as go
from scipy.stats import gaussian_kde
from sklearn.preprocessing import StandardScaler

# Pessoal

from useful.config import set_default_config
from useful.constants import RED, YELLOW, GREEN, PALETTE
from useful.plotly_tools import *
from useful.eda_useful import *

set_default_config()
scaler = StandardScaler()

In [2]:
# Carregando o dataset

with open('../data/processed/german-credit-data.pkl', 'rb') as file: 
    df = pickle.load(file)

# Outliers

Outliers está em primeira execução, pois a definição da análise exploratória foi baseada em plotar distribuições para verificar o método utilizado para encontrar os outliers em primeiro lugar.

Depois, verificamos a relevância dos outliers para decidir a relevância.

A distruição das colunas númericas do dataset não é normal, então iremos utilizar o método de IQR para remover outliers.

In [3]:
# Criando listas de features para a calculadora de IQR

df_iqr = df.copy()

EDA = EDAbasics(df_iqr)
EDA.separate_data('Target')
EDA.features_categories()

# Features fora do objeto

numerical_featuresiqr, categorical_featuresiqr, boolean_featuresiqr, heuristic_featuresiqr = EDA.numerical_features, EDA.categorical_features, EDA.boolean_features, EDA.heuristic_features

iqr = IQR_Calc(df_iqr, numerical_featuresiqr)
iqr.execute()
iqr.summary()

# Obtendo somente os outliers (linhas únicas)
outliers_iqr = pd.DataFrame(iqr.get_outliers('outliers_iqr'))

outliers_iqr

Calculando outliers com IQR
------------------------------
Coluna: duration
Outliers: 70
------------------------------
Coluna: credit_amount
Outliers: 72
------------------------------
Coluna: age
Outliers: 23
------------------------------
Total de outliers encontrados 138
Total de linhas no DataFrame com outliers 138


,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,Target
0,<0,6,critical/other existing credit,radio/tv,1169,no known savings,>=7,4,male single,none,4,real estate,67,none,own,2,skilled,1,True,True,good
1,0<=X<200,48,existing paid,radio/tv,5951,<100,1<=X<4,2,female div/dep/mar,none,2,real estate,22,none,own,1,skilled,1,False,True,bad
2,no checking,36,existing paid,education,9055,no known savings,1<=X<4,2,male single,none,4,no known property,35,none,for free,1,unskilled resident,2,True,True,good
3,<0,48,existing paid,business,4308,<100,<1,3,female div/dep/mar,none,4,life insurance,24,none,rent,1,skilled,1,False,True,bad
4,<0,30,no credits/all paid,business,8072,no known savings,<1,2,male single,none,3,car,25,bank,own,3,skilled,1,False,True,good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,0<=X<200,30,critical/other existing credit,furniture/equipment,8386,<100,4<=X<7,2,male single,none,2,life insurance,49,none,own,1,skilled,1,False,True,bad
134,no checking,48,existing paid,business,4844,<100,unemployed,3,male single,none,2,car,33,bank,rent,1,high qualif/self emp/mgmt,1,True,True,bad
135,<0,36,existing paid,used car,8229,<100,1<=X<4,2,male single,none,2,life insurance,26,none,own,1,skilled,2,False,True,bad
136,<0,45,existing paid,radio/tv,1845,<100,1<=X<4,4,male single,none,4,no known property,23,none,for free,1,skilled,1,True,True,bad


Após análise, foi identificado que os outliers detectados não têm relevância, sendo dados importantes. As colunas mencionadas pela calculadora de IQR não têm extremos e 

# Tratamento

Nesta etapa, é realizado um tratamento básico onde: Eliminamos colunas flags; Renomearemos a coluna 'class' para 'Target' e definimos as features.

Em segundo lugar, realizamos o mapeamento reverso das variáveis para melhor legibilidade e entendimento no EDA a partir dos dicionários criados no Pre-Processing para 'Target' e 'Features'.

In [4]:
# Renomeando a coluna 'class' para 'Target'
df.rename(columns={'class': 'Target'}, inplace=True)

# Separando features
features = df.drop(columns=['Target'])

# Análise Exploratória

## Target

### Proporção

In [5]:
# Contando a quantidade de valores e normalizando-os
target_prop = df['Target'].value_counts(ascending=True, normalize=True)

# Fixando eixos
x_fig1 = target_prop.index
y_fig1 = target_prop.values * 100 # Convertendo para porcentagem

# Criando o gráfico e exportando
fig1 = go.Figure()
fig1.add_trace(go.Bar(x=x_fig1, y=y_fig1, text=[f'{np.round(yy, 2)}%' for yy in y_fig1],
                      marker=dict(color=y_fig1, colorscale=[[0, RED], [0.5, YELLOW], [1, GREEN]])))
fig1.update_layout(title='Proporção das Classes', yaxis_title='Proporção (%)', xaxis_title='Classe',
                   height=600, autosize=True)
export_fig(fig1, 'target_prop', '../figs/eda/target_prop')
fig1.show()

## Features

### Definição

Separando/Definindo os tipos de dados das features e exportando para dicionários, para melhor usabilidade

In [6]:
# Usamos EDAbasics para separar os dados em features e target, além de categorizar as features
# e separá-las em numéricas, categóricas, booleanas e heurísticas

EDA_df = EDAbasics(df)
EDA_df.separate_data('Target')
EDA_df.features_categories()

# Colunas separadas por features em dicionários

features_dicts = {
    "numerical_features": {col: EDA_df.features[col] for col in EDA_df.numerical_features},
    "categorical_features": {col: EDA_df.features[col] for col in EDA_df.categorical_features},
    "boolean_features": {col: EDA_df.features[col] for col in EDA_df.boolean_features},
    "heuristic_features": {col: EDA_df.features[col] for col in EDA_df.heuristic_features}
}

# Criando variáveis externas para facilitar o acesso às features separadas

numerical_features, categorical_features, boolean_features, heuristic_features = EDA_df.numerical_features, EDA_df.categorical_features, EDA_df.boolean_features, EDA_df.heuristic_features

# Salvando para pickle

with open('../data/processed/features_dicts.pkl', 'wb') as file:
    pickle.dump(features_dicts, file)

In [7]:
categorical_features

['checking_status',
 'credit_history',
 'purpose',
 'savings_status',
 'employment',
 'personal_status',
 'other_parties',
 'property_magnitude',
 'other_payment_plans',
 'housing',
 'job']

### Distribuição

Linha Azul: Média
Linha vermelha: Mediana

#### Features Numéricas

In [8]:
# Distribuição de features

# Bins para o histograma
n = len(df)
bins = np.sqrt(n)

# Iteração sobre as features numéricas
for col in numerical_features:
    x_vals = df[col]

    fig2 = go.Figure()
    fig2.add_trace(go.Histogram(
        x=x_vals,
        xbins=dict(
            start=x_vals.min(),
            end=x_vals.max(),
            size=(x_vals.max() - x_vals.min()) / bins
        ),
        marker=dict(line=dict(color='black', width=1))
    ))
    fig2.add_vline(x=x_vals.mean(), line=dict(color='blue', dash='dash'), name='Média')
    fig2.add_vline(x=x_vals.median(), line=dict(color='red', dash='dot'), name='Mediana')
    fig2.update_layout(
        title=f'Distribuição {col}',
        yaxis_title='Frequência',
        xaxis_title=f'{col}',
        height=600,
        autosize=True
    )
    export_fig(fig2, f'dist_{col}', '../figs/eda/feat_dist')
    fig2.show()

### Proporção

#### Features Categóricas

Iteração pelas Features Categóricas para gerar os gráficos

In [ ]:
for col in categorical_features:
    x_vals = df[col]
    
    # Contar os valores únicos e suas proporções
    counts = x_vals.value_counts()
    total_count = len(df[col])
    counts_percentage = (counts / total_count) * 100
    x_vals = counts.index
    y_vals = counts_percentage.values
    
    fig3 = go.Figure()
    fig3.add_trace(go.Bar(x=x_vals, y=y_vals, text=[f'{np.round(yy, 2)}%' for yy in y_vals],
                      marker=dict(color=y_vals, colorscale=[[0, RED], [0.5, YELLOW], [1, GREEN]])))
    fig3.update_layout(title=f'Proporção {col}', yaxis_title='Proporção (%)', xaxis_title=f'{col}',
                     height=600, autosize=True)
    export_fig(fig3, f'feature_prop_{col}', '../figs/eda/feat_prop')
    fig3.show()

#### Features Heurísticas

Iteração pelas Features Heurísticas para gerar os gráficos

In [ ]:
for col in heuristic_features:
    x_vals = df[col]
    y_vals = df[col].value_counts(ascending=True).values
    
    counts = x_vals.value_counts()
    total_count = len(df[col])
    counts_percentage = (counts / total_count) * 100
    x_vals = counts.index
    y_vals = counts_percentage.values
    
    fig4 = go.Figure()
    fig4.add_trace(go.Bar(x=x_vals, y=y_vals, text=[f'{np.round(yy, 2)}%' for yy in y_vals],
                      marker=dict(color=y_vals, colorscale=[[0, RED], [0.5, YELLOW], [1, GREEN]])))
    fig4.update_layout(title=f'Proporção {col}', yaxis_title='Proporção (%)', xaxis_title=f'{col}',
                        height=600, autosize=True)
    # eixo x só deve rotulos para com algum valor
    fig4.update_xaxes(tickvals=x_vals[x_vals > 0])
    export_fig(fig4, f'feature_prop_{col}', '../figs/eda/feat_prop')
    fig4.show()

Iteração pelas Features Booleanas para gerar os gráficos

In [ ]:
for col in boolean_features:
    x_vals = df[col]
    
    # Count
    counts = x_vals.value_counts(ascending=True).values

    total_count = len(x_vals)
    counts_percentage = (counts / total_count) * 100

    fig5 = go.Figure()
    fig5.add_trace(go.Bar(x=x_vals.unique(), y=counts_percentage, text=[f'{np.round(yy, 2)}%' for yy in counts_percentage],
                      marker=dict(color=counts_percentage, colorscale=[[0, RED], [0.5, YELLOW], [1, GREEN]])))
    fig5.update_layout(title=f'Proporção {col}', yaxis_title='Proporção (%)', xaxis_title=f'{col}',
                   height=600, autosize=True)
    # retornando valores do dicionario de features_maps
    export_fig(fig5, f'feature_prop_{col}', '../figs/eda/feat_prop')
    fig5.show()

### Correlação

In [ ]:
# Retornando colunas booleanas

df['foreign_worker'] = df['foreign_worker'].astype(int)
df['own_telephone'] = df['own_telephone'].astype(int)

In [ ]:
cheatmap = CHeatmap(df)
fig = cheatmap.plot()
fig.show()

In [15]:
top = pd.DataFrame(cheatmap.top_correlations(n=30, threshold=0.15))

top

,Var1,Var2,Correlation
23,duration,credit_amount,0.62
54,credit_history,existing_credits,-0.39
337,job,own_telephone,-0.37
19,checking_status,Target,0.30
253,age,housing,-0.30
...,...,...,...
37,duration,own_telephone,0.16
30,duration,property_magnitude,-0.16
51,credit_history,age,-0.16
33,duration,housing,-0.16
